In [45]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, dayofweek
from pyspark.sql.types import IntegerType, TimestampType as Ts
from pyspark.sql.functions import monotonically_increasing_id

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Creating the Spark session
spark = SparkSession \
    .builder \
    .config('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:2.7.0') \
    .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Creating variables pointing to where data is stored
input_data = 's3a://udacity-dend/'
output_data = 's3://udacity-scb-proj2/result/'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
song_data = input_data + 'song_data/*/*/*/*.json'
log_data = input_data + 'log_data/*/*/*.json'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
song_data

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3a://udacity-dend/song_data/*/*/*/*.json'

In [6]:
log_data

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3a://udacity-dend/log_data/*/*/*.json'

## Processing the song data

In [7]:
df = spark.read.json(song_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
print(type(df))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pyspark.sql.dataframe.DataFrame'>

In [9]:
# Let us view the schema of the songs dataset
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)

In [10]:
df.take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(artist_id='AR4T2IF1187B9ADBB7', artist_latitude=63.96027, artist_location='<a href="http://billyidol.net" onmousedown=\'UntrustedLink.bootstrap($(this), "fc44f8f60d13ab68c56b3c6709c6d670", event)\' target="_blank" rel="nofollow">http://billyidol.net</a>', artist_longitude=10.22442, artist_name='Billy Idol', duration=233.22077, num_songs=1, song_id='SOVIYJY12AF72A4B00', title='The Dead Next Door (Digitally Remastered 99)', year=1983), Row(artist_id='AR4T2IF1187B9ADBB7', artist_latitude=63.96027, artist_location='<a href="http://billyidol.net" onmousedown=\'UntrustedLink.bootstrap($(this), "fc44f8f60d13ab68c56b3c6709c6d670", event)\' target="_blank" rel="nofollow">http://billyidol.net</a>', artist_longitude=10.22442, artist_name='Billy Idol', duration=287.92118, num_songs=1, song_id='SOVYXYL12AF72A3373', title='Rebel Yell (1999 Digital Remaster)', year=1983), Row(artist_id='ARQ846I1187B9A7083', artist_latitude=None, artist_location='', artist_longitude=None, artist_name='Yvonne S. M

In [11]:
# extract columns to create songs table
songs_table = df.select(['song_id','title','artist_id','year','duration'])\
                    .dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
songs_table.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(song_id='SONKVFM12AB01875C9', title='Amor De Madre/Mi Niña Cambió/Un Chi Chi Medley', artist_id='ARL4W6Q1187FB4713C', year=0, duration=639.13751)

In [13]:
path_song = os.path.join(output_data, "songs")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
path_song

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3://udacity-scb-proj2/result/songs'

In [15]:
songs_table.write.parquet(path_song, partitionBy=("year","artist_id"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'path s3://udacity-scb-proj2/result/songs already exists.;'
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/readwriter.py", line 841, in parquet
    self._jwrite.parquet(path)
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.AnalysisException: 'path s3://udacity-scb-proj2/result/songs already exists.;'



In [16]:
# extract columns to create artists table
# extract columns to create artists table
artists_table = df.select(['artist_id','artist_name','artist_location',
                               'artist_latitude', 'artist_longitude'])\
                      .dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
# write artists table to parquet files
path_artists = os.path.join(output_data, "artists")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
path_artists

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3://udacity-scb-proj2/result/artists'

In [19]:
artists_table.write.parquet(path_artists)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'path s3://udacity-scb-proj2/result/artists already exists.;'
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/readwriter.py", line 841, in parquet
    self._jwrite.parquet(path)
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.AnalysisException: 'path s3://udacity-scb-proj2/result/artists already exists.;'



## Processing the log data

In [20]:
log_data

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3a://udacity-dend/log_data/*/*/*.json'

In [21]:
log_data = os.path.join(input_data, "log_data/*/*/*.json")
log_data

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3a://udacity-dend/log_data/*/*/*.json'

In [22]:
df = spark.read.json(log_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
# filter by actions for song plays
df = df.where(df.page == "NextSong")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
df.head(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(artist='Harmonia', auth='Logged In', firstName='Ryan', gender='M', itemInSession=0, lastName='Smith', length=655.77751, level='free', location='San Jose-Sunnyvale-Santa Clara, CA', method='PUT', page='NextSong', registration=1541016707796.0, sessionId=583, song='Sehr kosmisch', status=200, ts=1542241826796, userAgent='"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"', userId='26'), Row(artist='The Prodigy', auth='Logged In', firstName='Ryan', gender='M', itemInSession=1, lastName='Smith', length=260.07465, level='free', location='San Jose-Sunnyvale-Santa Clara, CA', method='PUT', page='NextSong', registration=1541016707796.0, sessionId=583, song='The Big Gundown', status=200, ts=1542242481796, userAgent='"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"', userId='26'), Row(artist='Train', auth='Logged In', 

In [25]:
# extract columns for users table    
users_table = df.select(['userId','firstName','lastName','gender','level'])\
                    .dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
# write users table to parquet files
path_users = os.path.join(output_data, "users")
users_table.write.parquet(path_users)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'path s3://udacity-scb-proj2/result/users already exists.;'
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/readwriter.py", line 841, in parquet
    self._jwrite.parquet(path)
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.AnalysisException: 'path s3://udacity-scb-proj2/result/users already exists.;'



In [27]:
# create timestamp column from original timestamp column
get_timestamp = udf(lambda x: x/1000, IntegerType())
df = df.withColumn('timestamp', get_timestamp(df.ts))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
df.head(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(artist='Harmonia', auth='Logged In', firstName='Ryan', gender='M', itemInSession=0, lastName='Smith', length=655.77751, level='free', location='San Jose-Sunnyvale-Santa Clara, CA', method='PUT', page='NextSong', registration=1541016707796.0, sessionId=583, song='Sehr kosmisch', status=200, ts=1542241826796, userAgent='"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"', userId='26', timestamp=None), Row(artist='The Prodigy', auth='Logged In', firstName='Ryan', gender='M', itemInSession=1, lastName='Smith', length=260.07465, level='free', location='San Jose-Sunnyvale-Santa Clara, CA', method='PUT', page='NextSong', registration=1541016707796.0, sessionId=583, song='The Big Gundown', status=200, ts=1542242481796, userAgent='"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"', userId='26', timestamp=None), Row(ar

In [29]:
get_datetime = udf(lambda x: datetime.fromtimestamp(x), Ts())
df = df.withColumn('datetime', get_datetime(df.timestamp))
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
df.head(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(artist='Harmonia', auth='Logged In', firstName='Ryan', gender='M', itemInSession=0, lastName='Smith', length=655.77751, level='free', location='San Jose-Sunnyvale-Santa Clara, CA', method='PUT', page='NextSong', registration=1541016707796.0, sessionId=583, song='Sehr kosmisch', status=200, ts=1542241826796, userAgent='"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"', userId='26', timestamp=None, datetime=datetime.datetime(2018, 11, 15, 0, 30, 26, 796000)), Row(artist='The Prodigy', auth='Logged In', firstName='Ryan', gender='M', itemInSession=1, lastName='Smith', length=260.07465, level='free', location='San Jose-Sunnyvale-Santa Clara, CA', method='PUT', page='NextSong', registration=1541016707796.0, sessionId=583, song='The Big Gundown', status=200, ts=1542242481796, userAgent='"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1

In [32]:
# extract columns to create time table
time_table = df.select(
        col('datetime').alias('start_time'),
        hour('datetime').alias('hour'),
        dayofmonth('datetime').alias('day'),
        weekofyear('datetime').alias('week'),
        month('datetime').alias('month'),
        year('datetime').alias('year'),
        dayofweek('datetime').alias('weekday')
    ).dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
path_tt = os.path.join(output_data, "time")
path_tt

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3://udacity-scb-proj2/result/time'

In [34]:
time_table.write.parquet(path_tt, partitionBy = ["year","month"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'path s3://udacity-scb-proj2/result/time already exists.;'
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/readwriter.py", line 841, in parquet
    self._jwrite.parquet(path)
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.AnalysisException: 'path s3://udacity-scb-proj2/result/time already exists.;'



In [35]:
time_table.head(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(start_time=datetime.datetime(2018, 11, 15, 14, 9, 23, 796000), hour=14, day=15, week=46, month=11, year=2018, weekday=5), Row(start_time=datetime.datetime(2018, 11, 15, 15, 24, 7, 796000), hour=15, day=15, week=46, month=11, year=2018, weekday=5), Row(start_time=datetime.datetime(2018, 11, 15, 16, 31, 27, 796000), hour=16, day=15, week=46, month=11, year=2018, weekday=5), Row(start_time=datetime.datetime(2018, 11, 15, 19, 22, 32, 796000), hour=19, day=15, week=46, month=11, year=2018, weekday=5), Row(start_time=datetime.datetime(2018, 11, 21, 17, 40, 8, 796000), hour=17, day=21, week=47, month=11, year=2018, weekday=4)]

In [36]:
# read in song data to use for songplays table
song_df = spark.read.parquet(output_data+"songs")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
readwritersong_df.head(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(song_id='SODUMDU12AC468A22B', title="We're Skrewed (Otto Von Schirach My Supernatural Motorcycle Gang Will Knife U Open Piss & Shit All Over Your Organic Starbucks =iPhone Bank Account Lifeless Lifestyle Remix)", duration=249.5473, year=0, artist_id='ARHOSMU1242078130D'), Row(song_id='SOTCIHX12A8C13DDD2', title='Finally_ as that blazing sun shone down upon us_ did we know that true enemy was the voice of blind idolatry; and only then did we begin to think for ourselves.', duration=483.34322, year=2006, artist_id='ARYOIZG1187FB41E30'), Row(song_id='SOPSXLI12A6D4FA418', title='Practical Cats - An entertainment for speaker and orchestra (1954) (1998 Digital Remaster): Old Deuteronomy (Andante teneramente)', duration=251.03628, year=0, artist_id='ARR79V31187FB5B96E'), Row(song_id='SOUFBFK12A8C13D668', title="String Quartets Op. 74 (Hob. III: 72-74)  'Apponyi'_ String Quartet in C major Op. 74 No. 1: III.     Menuetto (Allegretto)", duration=348.60363, year=0, artist_id='ARAILTA11F4C84

In [39]:
df = df.join(song_df, song_df.title == df.song)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
df.head(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(artist='Lionel Rogg', auth='Logged In', firstName='Mohammad', gender='M', itemInSession=13, lastName='Rodriguez', length=172.38159, level='paid', location='Sacramento--Roseville--Arden-Arcade, CA', method='PUT', page='NextSong', registration=1540511766796.0, sessionId=744, song='Die Kunst der Fuge_ BWV 1080 (2007 Digital Remaster): Contrapunctus XVII - Inversus', status=200, ts=1542788743796, userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId='88', timestamp=None, datetime=datetime.datetime(2018, 11, 21, 8, 25, 43, 796000), song_id='SOCHPTV12A6BD53113', title='Die Kunst der Fuge_ BWV 1080 (2007 Digital Remaster): Contrapunctus XVII - Inversus', duration=172.38159, year=0, artist_id='ARN8NCB1187FB49652'), Row(artist='MC Lars', auth='Logged In', firstName='Layla', gender='F', itemInSession=81, lastName='Griffin', length=222.87628, level='paid', location='Lake Havasu City-Kingman, AZ', method=

In [46]:
df = df.withColumn('songplay_id', monotonically_increasing_id())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [62]:
df = df.withColumn("month", month(df.datetime))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
df.head(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(artist='Lionel Rogg', auth='Logged In', firstName='Mohammad', gender='M', itemInSession=13, lastName='Rodriguez', length=172.38159, level='paid', location='Sacramento--Roseville--Arden-Arcade, CA', method='PUT', page='NextSong', registration=1540511766796.0, sessionId=744, song='Die Kunst der Fuge_ BWV 1080 (2007 Digital Remaster): Contrapunctus XVII - Inversus', status=200, ts=1542788743796, userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId='88', timestamp=None, datetime=datetime.datetime(2018, 11, 21, 8, 25, 43, 796000), song_id='SOCHPTV12A6BD53113', title='Die Kunst der Fuge_ BWV 1080 (2007 Digital Remaster): Contrapunctus XVII - Inversus', duration=172.38159, year=0, artist_id='ARN8NCB1187FB49652', songplay_id=0, month=11), Row(artist='MC Lars', auth='Logged In', firstName='Layla', gender='F', itemInSession=81, lastName='Griffin', length=222.87628, level='paid', location='Lake Havasu C

In [64]:
songplays_table = df['songplay_id', 'year', 'month', 'userId','level', \
                      'song_id', 'artist_id', 'sessionId', 'location', 'userAgent']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
songplays_table.head(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(songplay_id=0, year=0, month=11, userId='88', level='paid', song_id='SOCHPTV12A6BD53113', artist_id='ARN8NCB1187FB49652', sessionId=744, location='Sacramento--Roseville--Arden-Arcade, CA', userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"'), Row(songplay_id=17179869184, year=2009, month=11, userId='24', level='paid', song_id='SOHRHCN12AB018B0F4', artist_id='ARHQBRZ1187FB3BDA2', sessionId=984, location='Lake Havasu City-Kingman, AZ', userAgent='"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"')]

In [67]:
path_songplays = os.path.join(output_data, 'songplays')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
path_songplays

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3://udacity-scb-proj2/result/songplays'

In [ ]:
songplays_table.write.parquet(path_songplays, partitionBy=["year","month"])


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…